# Evaluation Report - AI Palatal Rugoscopy

1. Import libraries to run evaluation.

In [ ]:
from src.architectures.segmenter_maskrcnn import MaskRCNNSegmenter
from src.engine.trainer import SupervisedTrainer
from src.training.train import load_dataset
from src.dataset.preprocessing import CocoPreprocessing
from src.dataset.composer import OrderedCompose
import matplotlib.pyplot as plt
import numpy as np
import torch
import gc

2. Load dataset and trained model

In [ ]:
TEST_DATASET_PATH = "/home/joaoherrera/data/rugae/manual/images"
TEST_DATASET_ANNOTATIONS_PATH = "/home/joaoherrera/data/rugae/manual/annotations/validation_annotations.json"
MODEL_PATH = "/home/joaoherrera/data/rugae/manual/output/train/full/checkpoint.pth"
SEED = 2183648025
device = torch.device("cuda")

In [ ]:
pp_funcs = OrderedCompose([CocoPreprocessing.resize_to_target], resize_target=1024)
test_dataset, test_dataloader = load_dataset(
    images_path=TEST_DATASET_PATH,
    annotations_path=TEST_DATASET_ANNOTATIONS_PATH,
    augmentations_funcs=pp_funcs,
    preprocessing_funcs=None,
    batch_size=1,
    shuffle=True,
    seed=SEED,
)

model = MaskRCNNSegmenter(MODEL_PATH, num_classes=len(test_dataset.categories.keys()))
successfully_loaded = model.load()
model = model.to(device)

print("Loaded model:", successfully_loaded)

3. Run PyCocoTools' evaluation function

In [ ]:
supervised_trainer = SupervisedTrainer("cuda:0", model)
supervised_trainer.coco_eval(test_dataloader)
supervised_trainer.evaluate(test_dataloader)

4. Run inference

In [ ]:

NUMBER_OF_SAMPLES = 1
random_samples_indexes = np.random.choice(len(test_dataset), NUMBER_OF_SAMPLES)

model = MaskRCNNSegmenter(MODEL_PATH, num_classes=len(test_dataset.categories.keys()))
model.load()
model.eval()

model = model.to(device)
with torch.no_grad():
    for idx in random_samples_indexes:
        sample = test_dataset[idx]
        image, _ = sample
        
        pred = model([image.to(device)], None)[0]
        final_mask = np.zeros(image.shape[1:3])
        
        for mask in pred["masks"]:
            mask = mask.detach().cpu().numpy()
            mask = np.squeeze(mask)
            mask[mask > 0.5] = 1
            mask[mask < 1.0] = 0
                
            final_mask += mask
        
        plt.imshow(final_mask)
        plt.show()

    model.cpu()
    del model
    gc.collect()
    torch.cuda.empty_cache()
